In [146]:
from bs4 import BeautifulSoup
import urllib2
import pandas as pd
import numpy as np
page = urllib2.urlopen('http://espn.go.com/nba/statistics/player/_/stat/assists/qualified/false/').read()
url_base = 'http://espn.go.com/nba/statistics/player/_/stat/%s/qualified/false/'
url_scoring_base = url_base % 'scoring'
url_assists_base = url_base % 'assists'

soup = BeautifulSoup(page)

soup.prettify()[0:1000]
table_div = soup.find(id = 'my-players-table')
table = table_div.find('table')
table_head = table.find(attrs={"class":'colhead'})
header_cols = table_head.findAll('td')
cols = []

for header_col in header_cols:
    val = header_col.string
    if val != 'RK':
        cols.append(val)
    if val == 'PLAYER':
        cols.append('POSITION')

table_rows = table.findAll('tr')

players_stats_dicts = []
players_stats_array = []
for row in table_rows:
    if row.attrs['class'][0]=='colhead':
        continue
    player_stats = []
    row_cols = row.find_all('td')
    col_vals = []
    player_col = row_cols[1]
    player_name = player_col.find('a').string
    player_position = player_col.contents[1]
    player_position = player_position.split(' ')[1]
    player_stats.append(player_name)
    player_stats.append(player_position)
    for i in range(2, len(row_cols)):
        stat = row_cols[i].string
        player_stats.append(stat)
    players_stats_array.append(player_stats)
    player_stats = dict(zip(cols, player_stats))
    players_stats_dicts.append(player_stats)

df = pd.DataFrame.from_dict(players_stats_dicts)
np_array = np.array(players_stats_array)
df = pd.DataFrame(np_array, columns=cols)
df.head()

,PLAYER,POSITION,TEAM,GP,MPG,AST,APG,TO,TOPG,AP48M,AST/TO
0,Russell Westbrook,PG,OKC,8,34.8,87,10.9,39,4.9,15.0,2.23
1,Ricky Rubio,PG,MIN,6,30.8,54,9.0,12,2.0,14.0,4.50
2,John Wall,PG,WSH,7,33.4,58,8.3,30,4.3,11.9,1.93
3,Ish Smith,PG,NO,7,26.9,57,8.1,13,1.9,14.6,4.38
4,Rajon Rondo,PG,SAC,8,31.9,62,7.8,27,3.4,11.7,2.30


In [57]:
#funtion to retreive the names of header columns, given a table element
def get_cols(table):
    header_cols = table.find(attrs={'class':'colhead'}).findAll('td')
    name_cols = []
    for element in header_cols:
        if (element.string == 'RK'):
            continue
        name_cols.append(element.string)
        if (element.string == 'PLAYER'):
            name_cols.append('POSITION')
    return name_cols


In [143]:
#function to retireve the actual data given a tabel element
def get_data(table, header_col):
    players = []
    rows = table.findAll('tr')
    for i in rows:
        if i.attrs['class'][0]=='colhead':
            continue
        player_stats = []
        row_cols = row.find_all('td')
        col_vals = []
        player_col = row_cols[1]
        player_name = player_col.find('a').string
        player_position = player_col.contents[1]
        player_position = player_position.split(' ')[1]
        player_stats.append(player_name)
        player_stats.append(player_position)
        for i in range(2, len(row_cols)):
            stat = row_cols[i].string
            player_stats.append(stat)
        players_stats_array.append(player_stats)
        #player_stats = dict(zip(cols, player_stats))
        #players_stats_dicts.append(player_stats)
    return players_stats_array
        
               

In [141]:
#iterate through various pages
pagination = np.arrange(1,402,40)
scoring = []
for i in pagination:
    page = urllib2.urlopen(base_url+'i').read()
    soup = BeautifulSoup(page)
    table = soup.find('tabe')
    s = get_data(table, header)
    scoring.extend(s)


In [145]:
#combine scoring and assists data sets into common data frame
df_combined = pd.DataFrame.merge(df_scoring, df_assists)
df_combined.head()

NameError: name 'df_scoring' is not defined